In [1]:
import pandas as pd

In [2]:
OUTPUT_FILENAME = 'Orgdata_20200109_ef20190701.csv'

In [3]:
itsDF = pd.read_csv("../data/WpaData_ef20190701.csv")

# データ数確認

In [4]:
itsDF.count()

EmployeeNo          2994
EmployeeName        2994
AdAccount1          2992
PersonId            2987
EffectiveDate       2994
ManagerId           2980
LevelDesignation    2994
Organization        2994
CompanyName         2994
HireDate            2994
EntryType           2994
EmploymentType      2994
CustomerCharge      2994
PreSales            2994
Available           2994
WorkingTime         2994
OverTime            2994
OnSite              2994
dtype: int64

# 欠損値の処理

### PersonIdがnullは削除する

#### PersonIdのnullを確認

In [5]:
itsDF[itsDF['PersonId'].isnull()].count()

EmployeeNo          7
EmployeeName        7
AdAccount1          6
PersonId            0
EffectiveDate       7
ManagerId           7
LevelDesignation    7
Organization        7
CompanyName         7
HireDate            7
EntryType           7
EmploymentType      7
CustomerCharge      7
PreSales            7
Available           7
WorkingTime         7
OverTime            7
OnSite              7
dtype: int64

#### PersonIdがnullの行を削除

In [6]:
itsDF.dropna(subset=['PersonId'], inplace=True)

#### PersonIdのnullが削除されたかを確認

In [7]:
itsDF[itsDF['PersonId'].isnull()].count()

EmployeeNo          0
EmployeeName        0
AdAccount1          0
PersonId            0
EffectiveDate       0
ManagerId           0
LevelDesignation    0
Organization        0
CompanyName         0
HireDate            0
EntryType           0
EmploymentType      0
CustomerCharge      0
PreSales            0
Available           0
WorkingTime         0
OverTime            0
OnSite              0
dtype: int64

### 全カラムのnullを最終確認する
#### ※AdAccount1とManagerIdのnullは無視

In [8]:
itsDF.isnull().sum()

EmployeeNo           0
EmployeeName         0
AdAccount1           1
PersonId             0
EffectiveDate        0
ManagerId           14
LevelDesignation     0
Organization         0
CompanyName          0
HireDate             0
EntryType            0
EmploymentType       0
CustomerCharge       0
PreSales             0
Available            0
WorkingTime          0
OverTime             0
OnSite               0
dtype: int64

#### AdAccount1のNullを確認

In [9]:
# 使用していないカラムなので確認だけ行い処理はしない
itsDF[itsDF['AdAccount1'].isnull()]

,EmployeeNo,EmployeeName,AdAccount1,PersonId,EffectiveDate,ManagerId,LevelDesignation,Organization,CompanyName,HireDate,EntryType,EmploymentType,CustomerCharge,PreSales,Available,WorkingTime,OverTime,OnSite
2988,95002,Karen Maya,NaN,karen.maya@jbs.com,07/01/2019,nobuyuki.jo@jbs.com,なし,JBSメキシコ,JBSメキシコ,09/01/2015,中途,正社員,0.0,0.0,0.0,0.0,0.0,0


# 重複の処理

### 完全に重複している行は削除する

#### 完全重複行の確認

In [10]:
itsDF[itsDF.duplicated()]

,EmployeeNo,EmployeeName,AdAccount1,PersonId,EffectiveDate,ManagerId,LevelDesignation,Organization,CompanyName,HireDate,EntryType,EmploymentType,CustomerCharge,PreSales,Available,WorkingTime,OverTime,OnSite


#### 完全重複行の削除

In [11]:
itsDF = itsDF.drop_duplicates()

#### 完全重複行の確認

In [12]:
itsDF[itsDF.duplicated()]

,EmployeeNo,EmployeeName,AdAccount1,PersonId,EffectiveDate,ManagerId,LevelDesignation,Organization,CompanyName,HireDate,EntryType,EmploymentType,CustomerCharge,PreSales,Available,WorkingTime,OverTime,OnSite


### 正社員と出向で重複しているケースは出向を残す

#### 登録されている社員数の確認

In [13]:
itsDF['EmployeeNo'].count()

2987

#### 社員番号が重複している人を確認する

In [14]:
itsDF[itsDF.duplicated(subset='EmployeeNo', keep=False)].sort_values(by='EmployeeNo')

,EmployeeNo,EmployeeName,AdAccount1,PersonId,EffectiveDate,ManagerId,LevelDesignation,Organization,CompanyName,HireDate,EntryType,EmploymentType,CustomerCharge,PreSales,Available,WorkingTime,OverTime,OnSite
2921,3959,城 信行,njo,nobuyuki.jo@jbs.com,07/01/2019,NaN,社長,JBS USA,JBS USA,08/01/2018,中途,出向者,0.0,0.0,0.0,0.0,0.0,0
2986,3959,城 信行,njo,nobuyuki.jo@jbs.com,07/01/2019,makita@jbs.com,Presidente / Director General,JBSメキシコ,JBSメキシコ,08/01/2018,中途,出向者,0.0,0.0,0.0,0.0,0.0,0
2931,90045,岡田 一郎,iokada,ichiro.okada@jbs.com,07/01/2019,isamu.kakitani@jbs.com,なし,Delivery Division ソリューション部 ERP,JBS USA,10/12/2015,中途,正社員,0.0,0.0,0.0,0.0,0.0,0
2987,90045,岡田 一郎,iokada,ichiro.okada@jbs.com,07/01/2019,nobuyuki.jo@jbs.com,なし,JBSメキシコ,JBSメキシコ,10/12/2015,中途,正社員,0.0,0.0,0.0,0.0,0.0,0


#### EmploymentTypeが'出向者'と'正社員'で重複している場合
#### ひとまず重複データを別のデータフレームに格納

In [15]:
dupdataDF = itsDF[itsDF.duplicated(subset='EmployeeNo', keep=False)].sort_values(by='EmployeeNo')

In [16]:
select_type_for_dupdataDF = dupdataDF[dupdataDF['EmploymentType'] == '出向者']

#### 上記以外の理由で重複している場合
#### データ削除

In [17]:
# メキシコ行を削除(重複人数分)
# indexを指定する
itsDF = itsDF.drop(2986)
itsDF = itsDF.drop(2987)

#### 社員番号が重複している人がいない事を確認する

In [18]:
itsDF[itsDF.duplicated(subset='EmployeeNo', keep=False)].sort_values(by='EmployeeNo')

,EmployeeNo,EmployeeName,AdAccount1,PersonId,EffectiveDate,ManagerId,LevelDesignation,Organization,CompanyName,HireDate,EntryType,EmploymentType,CustomerCharge,PreSales,Available,WorkingTime,OverTime,OnSite


#### 重複処理後の登録されている社員数の確認

In [19]:
itsDF['EmployeeNo'].count()

2985

# EmployeeNoをハッシュ化

In [20]:
import hashlib

### ハッシュ化用関数定義

In [21]:
def to_hash(data):
    salt = 'JBS'
    add_salt_data = salt + str(data)
    return hashlib.sha256(add_salt_data.encode('UTF-8')).hexdigest()

#### EmployeeNoにハッシュ化関数を適用して値を入れ替える

In [22]:
itsDF['hashedEmployeeNo'] = itsDF['EmployeeNo'].map(to_hash)

# TimeZoneを設定

#### CompanyNameの値からTimeZoneカラムを作成する

In [23]:
itsDF['CompanyName'].unique()

array(['JBS', 'JBS USA', 'JBSシンガポール', 'JBS上海', 'JBSメキシコ'], dtype=object)

TimeZone  
- JBS: 東京標準時[Central Standard Time (Mexico)]  
- JBS USA: 太平洋標準時[Pacific Standard Time]  
- JBSシンガポール: シンガポール標準時[Singapore Standard Time]  
- JBS上海: 中国標準時[China Standard Time]
- JBSメキシコ: 中部標準時 (メキシコ)[Central Standard Time (Mexico)]
- JBST: ※2019/?まで存在

In [24]:
import numpy as np
itsDF['TimeZone'] = np.NAN

In [25]:
itsDF.loc[itsDF['CompanyName'] == 'JBS', 'TimeZone'] = 'Tokyo Standard Time'
itsDF.loc[itsDF['CompanyName'] == 'JBS USA', 'TimeZone'] = 'Pacific Standard Time'
itsDF.loc[itsDF['CompanyName'] == 'JBSシンガポール', 'TimeZone'] = 'Singapore Standard Time'
itsDF.loc[itsDF['CompanyName'] == 'JBS上海', 'TimeZone'] = 'China Standard Time'
itsDF.loc[itsDF['CompanyName'] == 'JBSメキシコ', 'TimeZone'] = 'Central Standard Time (Mexico)'

# OrganizationからDiv(本部レベル)とDept(部レベル)を作成

In [26]:
itsDF['Organization'].unique()

array(['JBS', '内部監査室', '経営管理室', '財務経理部', '財務経理部 経理課', '財務経理部 財務課',
       '経営企画部', '品質管理室 ISMS／SM推進', '品質管理室 PM推進', 'リスク管理室', '人事部',
       '人事部 人事課', '人事部 採用1課', '人事部 採用2課', '人事部 ヘルスケアセンター', '総務部',
       '総務部 総務課', '総務部 法務課', "総務部 Lucy's課", '業務部 業務課', '業務部 購買課',
       '業務部 物流課', '情報システム本部 インフラストラクチャサービス部',
       '情報システム本部 インフラストラクチャサービス部 ITデザイン課',
       '情報システム本部 インフラストラクチャサービス部 ITサービス課', '情報システム本部 ビジネスサービス部',
       '情報システム本部 ビジネスサービス部 インプリメント課', '情報システム本部 ビジネスサービス部 マネジメント課',
       '社会システムソリューション室', '事業企画本部', '事業企画本部 事業企画本部付', '事業企画本部 総合企画部',
       '事業企画本部 総合企画部 パートナーアライアンス課', '事業企画本部 総合企画部 コミュニケーション課',
       '事業企画本部 海外事業推進室', '事業企画本部 人材開発部', '事業企画本部 人材開発部 2019年度新卒',
       '事業企画本部 MS統合サポートセンター テクニカルナレッジグループ',
       '事業企画本部 MS統合サポートセンター FastTrack推進グループ',
       '事業企画本部 MS統合サポートセンター ライセンスナレッジグループ', '事業企画本部 サービス企画部',
       '事業企画本部 サービス推進部', '事業企画本部 サービス推進部 シードアクセラレーショングループ',
       '事業企画本部 ビジネスインキュベーション部', '事業企画本部 ビジネスインキュベーション部 ISV推進グループ',
       '事業企画本部 ビジネスインキュベーション部 IP企画グル

### Organizationの最初のスペースまでをDivとする ※例外部署あり

In [27]:
# expand=FalseだとSeriesが返り値になる
tmp = itsDF['Organization'].str.split(' ', expand=True)

#### ほとんどの部署は最初のスペースまでを使用する

In [28]:
div_s = tmp.get(0)

In [29]:
# この後の工程(FunctionType)で利用するカラムを作成
itsDF['tmp_div'] = div_s

#### 例外部署処理(ITソリューション統括本部)

In [30]:
div_it_s = tmp[tmp.get(0) == 'ITソリューション統括本部'].get(1)

#### 処理によってnullが発生しないか確認

In [31]:
div_it_s[div_it_s.isnull()]

Series([], Name: 1, dtype: object)

In [32]:
div_s[div_s == 'ITソリューション統括本部'] = div_it_s

#### 例外部署処理(営業統括本部)

In [33]:
div_eg_s = tmp[tmp.get(0) == '営業統括本部'].get(1)

#### 処理によってnullが発生しないか確認

In [34]:
div_eg_s[div_eg_s.isnull()]

416    None
417    None
Name: 1, dtype: object

#### nullが発生する方の処理
※統括本部長等

In [35]:
div_eg_s[div_eg_s.isnull()] = '営業統括本部'

C:\Users\tharada\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\tharada\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [36]:
div_s[div_s == '営業統括本部'] = div_eg_s

In [37]:
itsDF['Div'] = div_s

In [38]:
itsDF['Div'].unique()

array(['JBS', '内部監査室', '経営管理室', '財務経理部', '経営企画部', '品質管理室', 'リスク管理室',
       '人事部', '総務部', '業務部', '情報システム本部', '社会システムソリューション室', '事業企画本部',
       '営業統括本部', '営業統括本部付', '第1営業本部', '第2営業本部', '第3営業本部', 'パートナー営業本部',
       '中部・西日本営業本部', 'ソリューション営業部', '営業サポート室', 'ITソリューション統括本部付',
       'デジタルイノベーション推進室', '金融システム本部', 'エンタープライズシステム本部', 'テクノロジーソリューション本部',
       'テクニカルパートナー本部', 'ビジネスソリューション本部', 'サービス＆サポート本部', '技術管理室',
       'Administration', 'Delivery', 'JBSAP', 'JBS上海', '技術部', 'サービス部',
       'JBSメキシコ'], dtype=object)

#### 例外部署処理(海外) ※CompanyNameが以下の行が対象
- JBS USA(USA)
- JBSシンガポール(SGP)
- JBS上海(CHN)
- JBSメキシコ(MEX)

In [39]:
itsDF.loc[itsDF.loc[:, 'CompanyName'] == 'JBS USA','Div'] = itsDF.loc[itsDF.loc[:,'CompanyName'] == 'JBS USA','Div'].map(lambda s: 'USA-' + s)

In [40]:
itsDF.loc[itsDF.loc[:, 'CompanyName'] == 'JBSシンガポール','Div'] = itsDF.loc[itsDF.loc[:,'CompanyName'] == 'JBSシンガポール','Div'].map(lambda s: 'SGP-' + s)

In [41]:
itsDF.loc[itsDF.loc[:, 'CompanyName'] == 'JBS上海','Div'] = itsDF.loc[itsDF.loc[:,'CompanyName'] == 'JBS上海','Div'].map(lambda s: 'CHN-' + s)

In [42]:
itsDF.loc[itsDF.loc[:, 'CompanyName'] == 'JBSメキシコ','Div'] = itsDF.loc[itsDF.loc[:,'CompanyName'] == 'JBSメキシコ','Div'].map(lambda s: 'MEX-' + s)

In [43]:
itsDF['Div'].unique()

array(['JBS', '内部監査室', '経営管理室', '財務経理部', '経営企画部', '品質管理室', 'リスク管理室',
       '人事部', '総務部', '業務部', '情報システム本部', '社会システムソリューション室', '事業企画本部',
       '営業統括本部', '営業統括本部付', '第1営業本部', '第2営業本部', '第3営業本部', 'パートナー営業本部',
       '中部・西日本営業本部', 'ソリューション営業部', '営業サポート室', 'ITソリューション統括本部付',
       'デジタルイノベーション推進室', '金融システム本部', 'エンタープライズシステム本部', 'テクノロジーソリューション本部',
       'テクニカルパートナー本部', 'ビジネスソリューション本部', 'サービス＆サポート本部', '技術管理室', 'USA-JBS',
       'USA-Administration', 'USA-Delivery', 'SGP-JBSAP', 'CHN-JBS上海',
       'CHN-技術部', 'CHN-サービス部', 'MEX-JBSメキシコ'], dtype=object)

#### OrganizationとDeptの対応辞書

In [44]:
DEPT_DICT = {
 'JBS': 'JBS',
 '内部監査室': '内部監査室',
 '経営管理室': '経営管理室',
 '財務経理部': '財務経理部',
 '財務経理部 経理課': '財務経理部',
 '財務経理部 財務課': '財務経理部',
 '経営企画部': '経営企画部',
 '品質管理室 ISMS／SM推進': '品質管理室',
 '品質管理室 PM推進': '品質管理室',
 'リスク管理室': 'リスク管理室',
 '人事部': '人事部',
 '人事部 人事課': '人事部',
 '人事部 採用1課': '人事部',
 '人事部 採用2課': '人事部',
 '人事部 ヘルスケアセンター': '人事部',
 '総務部': '総務部',
 '総務部 総務課': '総務部',
 '総務部 法務課': '総務部',
 "総務部 Lucy's課": '総務部',
 '業務部 業務課': '業務部',
 '業務部 購買課': '業務部',
 '業務部 物流課': '業務部',
 '情報システム本部 インフラストラクチャサービス部': '情報システム本部 インフラストラクチャサービス部',
 '情報システム本部 インフラストラクチャサービス部 ITデザイン課': '情報システム本部 インフラストラクチャサービス部',
 '情報システム本部 インフラストラクチャサービス部 ITサービス課': '情報システム本部 インフラストラクチャサービス部',
 '情報システム本部 ビジネスサービス部': '情報システム本部 ビジネスサービス部',
 '情報システム本部 ビジネスサービス部 インプリメント課': '情報システム本部 ビジネスサービス部',
 '情報システム本部 ビジネスサービス部 マネジメント課': '情報システム本部 ビジネスサービス部',
 '社会システムソリューション室': '社会システムソリューション室',
 '事業企画本部': '事業企画本部',
 '事業企画本部 事業企画本部付': '事業企画本部 事業企画本部付',
 '事業企画本部 総合企画部': '事業企画本部 総合企画部',
 '事業企画本部 総合企画部 パートナーアライアンス課': '事業企画本部 総合企画部',
 '事業企画本部 総合企画部 コミュニケーション課': '事業企画本部 総合企画部',
 '事業企画本部 海外事業推進室': '事業企画本部 海外事業推進室',
 '事業企画本部 人材開発部': '事業企画本部 人材開発部',
 '事業企画本部 人材開発部 2019年度新卒': '事業企画本部 人材開発部',
 '事業企画本部 MS統合サポートセンター テクニカルナレッジグループ': '事業企画本部 MS統合サポートセンター',
 '事業企画本部 MS統合サポートセンター FastTrack推進グループ': '事業企画本部 MS統合サポートセンター',
 '事業企画本部 MS統合サポートセンター ライセンスナレッジグループ': '事業企画本部 MS統合サポートセンター',
 '事業企画本部 サービス企画部': '事業企画本部 サービス企画部',
 '事業企画本部 サービス推進部': '事業企画本部 サービス推進部',
 '事業企画本部 サービス推進部 シードアクセラレーショングループ': '事業企画本部 サービス推進部',
 '事業企画本部 ビジネスインキュベーション部': '事業企画本部 ビジネスインキュベーション部',
 '事業企画本部 ビジネスインキュベーション部 ISV推進グループ': '事業企画本部 ビジネスインキュベーション部',
 '事業企画本部 ビジネスインキュベーション部 IP企画グループ': '事業企画本部 ビジネスインキュベーション部',
 '営業統括本部': '営業統括本部',
 '営業統括本部 営業統括本部付': '営業統括本部 営業統括本部付',
 '営業統括本部 第1営業本部': '営業統括本部 第1営業本部',
 '営業統括本部 第1営業本部 営業1部': '営業統括本部 第1営業本部 営業1部',
 '営業統括本部 第1営業本部 営業1部 営業1課': '営業統括本部 第1営業本部 営業1部',
 '営業統括本部 第1営業本部 営業1部 営業2課': '営業統括本部 第1営業本部 営業1部',
 '営業統括本部 第1営業本部 営業2部': '営業統括本部 第1営業本部 営業2部',
 '営業統括本部 第1営業本部 営業2部 営業1課': '営業統括本部 第1営業本部 営業2部',
 '営業統括本部 第1営業本部 営業2部 営業2課': '営業統括本部 第1営業本部 営業2部',
 '営業統括本部 第2営業本部': '営業統括本部 第2営業本部',
 '営業統括本部 第2営業本部 営業1部': '営業統括本部 第2営業本部 営業1部',
 '営業統括本部 第2営業本部 営業1部 営業1課': '営業統括本部 第2営業本部 営業1部',
 '営業統括本部 第2営業本部 営業1部 営業2課': '営業統括本部 第2営業本部 営業1部',
 '営業統括本部 第2営業本部 営業2部': '営業統括本部 第2営業本部 営業2部',
 '営業統括本部 第2営業本部 営業2部 営業1課': '営業統括本部 第2営業本部 営業2部',
 '営業統括本部 第2営業本部 営業2部 営業2課': '営業統括本部 第2営業本部 営業2部',
 '営業統括本部 第3営業本部': '営業統括本部 第3営業本部',
 '営業統括本部 第3営業本部 第3営業本部付': '営業統括本部 第3営業本部 第3営業本部付',
 '営業統括本部 第3営業本部 営業1部': '営業統括本部 第3営業本部 営業1部',
 '営業統括本部 第3営業本部 営業1部 営業1課': '営業統括本部 第3営業本部 営業1部',
 '営業統括本部 第3営業本部 営業1部 営業2課': '営業統括本部 第3営業本部 営業1部',
 '営業統括本部 第3営業本部 営業2部': '営業統括本部 第3営業本部 営業2部',
 '営業統括本部 第3営業本部 営業2部 営業1課': '営業統括本部 第3営業本部 営業2部',
 '営業統括本部 第3営業本部 営業3部': '営業統括本部 第3営業本部 営業3部',
 '営業統括本部 第3営業本部 営業3部 営業1課': '営業統括本部 第3営業本部 営業3部',
 '営業統括本部 第3営業本部 営業3部 営業2課': '営業統括本部 第3営業本部 営業3部',
 '営業統括本部 第3営業本部 営業3部 営業3課': '営業統括本部 第3営業本部 営業3部',
 '営業統括本部 パートナー営業本部': '営業統括本部 パートナー営業本部',
 '営業統括本部 パートナー営業本部 営業部': '営業統括本部 パートナー営業本部 営業部',
 '営業統括本部 パートナー営業本部 営業部 営業1課': '営業統括本部 パートナー営業本部 営業部',
 '営業統括本部 パートナー営業本部 営業部 営業2課': '営業統括本部 パートナー営業本部 営業部',
 '営業統括本部 中部・西日本営業本部': '営業統括本部 中部・西日本営業本部',
 '営業統括本部 中部・西日本営業本部 中部営業部': '営業統括本部 中部・西日本営業本部 中部営業部',
 '営業統括本部 中部・西日本営業本部 西日本営業部': '営業統括本部 中部・西日本営業本部 西日本営業部',
 '営業統括本部 中部・西日本営業本部 東京営業部': '営業統括本部 中部・西日本営業本部 東京営業部',
 '営業統括本部 中部・西日本営業本部 北海道営業部': '営業統括本部 中部・西日本営業本部 北海道営業部',
 '営業統括本部 中部・西日本営業本部 九州営業部': '営業統括本部 中部・西日本営業本部 九州営業部',
 '営業統括本部 ソリューション営業部': '営業統括本部 ソリューション営業部',
 '営業統括本部 ソリューション営業部 営業1課': '営業統括本部 ソリューション営業部',
 '営業統括本部 ソリューション営業部 営業2課': '営業統括本部 ソリューション営業部',
 '営業統括本部 ソリューション営業部 営業3課': '営業統括本部 ソリューション営業部',
 '営業統括本部 ソリューション営業部 MSライセンス課': '営業統括本部 ソリューション営業部',
 '営業統括本部 営業サポート室': '営業統括本部 営業サポート室',
 'ITソリューション統括本部 ITソリューション統括本部付': 'ITソリューション統括本部 ITソリューション統括本部付',
 'ITソリューション統括本部 デジタルイノベーション推進室': 'ITソリューション統括本部 デジタルイノベーション推進室',
 'ITソリューション統括本部 デジタルイノベーション推進室 デジタルコンサルティング推進': 'ITソリューション統括本部 デジタルイノベーション推進室',
 'ITソリューション統括本部 デジタルイノベーション推進室 ITアーキテクト推進': 'ITソリューション統括本部 デジタルイノベーション推進室',
 'ITソリューション統括本部 デジタルイノベーション推進室 PMO推進': 'ITソリューション統括本部 デジタルイノベーション推進室',
 'ITソリューション統括本部 金融システム本部': 'ITソリューション統括本部 金融システム本部',
 'ITソリューション統括本部 金融システム本部 金融システム1部': 'ITソリューション統括本部 金融システム本部 金融システム1部',
 'ITソリューション統括本部 金融システム本部 金融システム1部 アカウント第1グループ': 'ITソリューション統括本部 金融システム本部 金融システム1部',
 'ITソリューション統括本部 金融システム本部 金融システム1部 アカウント第2グループ': 'ITソリューション統括本部 金融システム本部 金融システム1部',
 'ITソリューション統括本部 金融システム本部 金融システム1部 アカウント第3グループ': 'ITソリューション統括本部 金融システム本部 金融システム1部',
 'ITソリューション統括本部 金融システム本部 金融システム1部 アカウント第4グループ': 'ITソリューション統括本部 金融システム本部 金融システム1部',
 'ITソリューション統括本部 金融システム本部 金融システム1部 アカウント第5グループ': 'ITソリューション統括本部 金融システム本部 金融システム1部',
 'ITソリューション統括本部 金融システム本部 金融システム2部': 'ITソリューション統括本部 金融システム本部 金融システム2部',
 'ITソリューション統括本部 金融システム本部 金融システム2部 アカウント第1グループ': 'ITソリューション統括本部 金融システム本部 金融システム2部',
 'ITソリューション統括本部 金融システム本部 金融システム2部 アカウント第2グループ': 'ITソリューション統括本部 金融システム本部 金融システム2部',
 'ITソリューション統括本部 金融システム本部 金融システム2部 アカウント第3グループ': 'ITソリューション統括本部 金融システム本部 金融システム2部',
 'ITソリューション統括本部 金融システム本部 金融システム2部 アカウント第4グループ': 'ITソリューション統括本部 金融システム本部 金融システム2部',
 'ITソリューション統括本部 金融システム本部 金融システム2部 アカウント第5グループ': 'ITソリューション統括本部 金融システム本部 金融システム2部',
 'ITソリューション統括本部 金融システム本部 金融システム3部': 'ITソリューション統括本部 金融システム本部 金融システム3部',
 'ITソリューション統括本部 金融システム本部 金融システム3部 アカウント第1グループ': 'ITソリューション統括本部 金融システム本部 金融システム3部',
 'ITソリューション統括本部 金融システム本部 金融システム3部 アカウント第2グループ': 'ITソリューション統括本部 金融システム本部 金融システム3部',
 'ITソリューション統括本部 金融システム本部 金融システム4部': 'ITソリューション統括本部 金融システム本部 金融システム4部',
 'ITソリューション統括本部 金融システム本部 金融システム4部 アカウント第1グループ': 'ITソリューション統括本部 金融システム本部 金融システム4部',
 'ITソリューション統括本部 金融システム本部 金融システム4部 アカウント第2グループ': 'ITソリューション統括本部 金融システム本部 金融システム4部',
 'ITソリューション統括本部 金融システム本部 金融システム4部 アカウント第3グループ': 'ITソリューション統括本部 金融システム本部 金融システム4部',
 'ITソリューション統括本部 エンタープライズシステム本部': 'ITソリューション統括本部 エンタープライズシステム本部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム1部 アカウント第1グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム1部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム1部 アカウント第2グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム1部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム1部 アカウント第3グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム1部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム1部 アカウント第4グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム1部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム2部': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム2部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム2部 アカウント第1グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム2部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム2部 アカウント第2グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム2部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部 アカウント第1グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部 アカウント第2グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部 アカウント第3グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部 アカウント第4グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部 アカウント第5グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム3部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム4部': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム4部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム4部 アカウント第1グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム4部',
 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム4部 アカウント第2グループ': 'ITソリューション統括本部 エンタープライズシステム本部 エンタープライズシステム4部',
 'ITソリューション統括本部 エンタープライズシステム本部 中部技術部 アカウント1グループ': 'ITソリューション統括本部 エンタープライズシステム本部 中部技術部',
 'ITソリューション統括本部 エンタープライズシステム本部 中部技術部 アカウント2グループ': 'ITソリューション統括本部 エンタープライズシステム本部 中部技術部',
 'ITソリューション統括本部 エンタープライズシステム本部 中部技術部 アカウント3グループ': 'ITソリューション統括本部 エンタープライズシステム本部 中部技術部',
 'ITソリューション統括本部 エンタープライズシステム本部 中部技術部 アカウント4グループ': 'ITソリューション統括本部 エンタープライズシステム本部 中部技術部',
 'ITソリューション統括本部 エンタープライズシステム本部 西日本技術部': 'ITソリューション統括本部 エンタープライズシステム本部 西日本技術部',
 'ITソリューション統括本部 エンタープライズシステム本部 西日本技術部 アカウント1グループ': 'ITソリューション統括本部 エンタープライズシステム本部 西日本技術部',
 'ITソリューション統括本部 エンタープライズシステム本部 西日本技術部 アカウント2グループ': 'ITソリューション統括本部 エンタープライズシステム本部 西日本技術部',
 'ITソリューション統括本部 エンタープライズシステム本部 西日本技術部 アカウント3グループ': 'ITソリューション統括本部 エンタープライズシステム本部 西日本技術部',
 'ITソリューション統括本部 テクノロジーソリューション本部': 'ITソリューション統括本部 テクノロジーソリューション本部',
 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部': 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部 アカウント1グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部 アカウント2グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部 CDグループ': 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部 xプロダクティビティ1グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部 xプロダクティビティ3グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 ナレッジコラボレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部': 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部',
 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部 1グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部',
 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部 2グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部',
 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部 3グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部',
 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部 4グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部',
 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部 5グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部',
 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部 6グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部',
 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部 7グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 クラウドプラットフォーム部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション1グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション2グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション3グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション4グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション5グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション6グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション7グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション8グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション9グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション10グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション11グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション12グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部 システムインテグレーション13グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 システムインテグレーション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部': 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部 クラウドソリューション1グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部 クラウドソリューション2グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部 セキュリティーソリューショングループ': 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部 ワークスタイルソリューション1グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部 ワークスタイルソリューション2グループ': 'ITソリューション統括本部 テクノロジーソリューション本部 プラットフォームソリューション部',
 'ITソリューション統括本部 テクノロジーソリューション本部 セキュアデザインセンター': 'ITソリューション統括本部 テクノロジーソリューション本部 セキュアデザインセンター',
 'ITソリューション統括本部 テクノロジーソリューション本部 テクノロジーデザインセンター': 'ITソリューション統括本部 テクノロジーソリューション本部 テクノロジーデザインセンター',
 'ITソリューション統括本部 テクニカルパートナー本部': 'ITソリューション統括本部 テクニカルパートナー本部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部 第1技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部 第2技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部 第3技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部 第4技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部 第5技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部 第6技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部 第7技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部 第8技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部 第9技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー1部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー2部': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー2部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー2部 第1技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー2部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー2部 第2技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー2部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー2部 第3技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー2部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー3部': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー3部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー3部 第4技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー3部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー3部 第7技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー3部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー3部 第8技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー3部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー4部': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー4部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー4部 第1技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー4部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー4部 第3技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー4部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー4部 第4技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー4部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー5部 第1技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー5部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー5部 第2技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー5部',
 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー5部 第3技術グループ': 'ITソリューション統括本部 テクニカルパートナー本部 テクニカルパートナー5部',
 'ビジネスソリューション本部': 'ビジネスソリューション本部',
 'ビジネスソリューション本部 ビジネスソリューション本部付': 'ビジネスソリューション本部 ビジネスソリューション本部付',
 'ビジネスソリューション本部 ビジネスアーキテクト室': 'ビジネスソリューション本部 ビジネスアーキテクト室',
 'ビジネスソリューション本部 テクノロジーアーキテクト室': 'ビジネスソリューション本部 テクノロジーアーキテクト室',
 'ビジネスソリューション本部 ソリューション開発室': 'ビジネスソリューション本部 ソリューション開発室',
 'ビジネスソリューション本部 ビジネスアプリケーション1部': 'ビジネスソリューション本部 ビジネスアプリケーション1部',
 'ビジネスソリューション本部 ビジネスアプリケーション1部 アプリケーションアーキテクト1グループ': 'ビジネスソリューション本部 ビジネスアプリケーション1部',
 'ビジネスソリューション本部 ビジネスアプリケーション1部 アプリケーションアーキテクト2グループ': 'ビジネスソリューション本部 ビジネスアプリケーション1部',
 'ビジネスソリューション本部 ビジネスアプリケーション1部 アプリケーションアーキテクト3グループ': 'ビジネスソリューション本部 ビジネスアプリケーション1部',
 'ビジネスソリューション本部 ビジネスアプリケーション1部 アプリケーションアーキテクト沖縄グループ': 'ビジネスソリューション本部 ビジネスアプリケーション1部',
 'ビジネスソリューション本部 ビジネスアプリケーション2部': 'ビジネスソリューション本部 ビジネスアプリケーション2部',
 'ビジネスソリューション本部 ビジネスアプリケーション2部 アプリケーションアーキテクト1グループ': 'ビジネスソリューション本部 ビジネスアプリケーション2部',
 'ビジネスソリューション本部 ビジネスアプリケーション2部 アプリケーションアーキテクト2グループ': 'ビジネスソリューション本部 ビジネスアプリケーション2部',
 'ビジネスソリューション本部 ビジネスアプリケーション2部 アプリケーションアーキテクト3グループ': 'ビジネスソリューション本部 ビジネスアプリケーション2部',
 'ビジネスソリューション本部 ERPソリューション部': 'ビジネスソリューション本部 ERPソリューション部',
 'ビジネスソリューション本部 ERPソリューション部 RPAソリューショングループ': 'ビジネスソリューション本部 ERPソリューション部',
 'ビジネスソリューション本部 ERPソリューション部 ビジネスオペレーション1G': 'ビジネスソリューション本部 ERPソリューション部',
 'ビジネスソリューション本部 ERPソリューション部 ビジネスオペレーション2G': 'ビジネスソリューション本部 ERPソリューション部',
 'ビジネスソリューション本部 ERPソリューション部 インプリメンテーショングループ': 'ビジネスソリューション本部 ERPソリューション部',
 'ビジネスソリューション本部 ERPソリューション部 ERP基盤グループ': 'ビジネスソリューション本部 ERPソリューション部',
 'ビジネスソリューション本部 ERPソリューション部 データアーキテクトグループ': 'ビジネスソリューション本部 ERPソリューション部',
 'ビジネスソリューション本部 CRMソリューション部': 'ビジネスソリューション本部 CRMソリューション部',
 'ビジネスソリューション本部 CRMソリューション部 インプリメンテーション1グループ': 'ビジネスソリューション本部 CRMソリューション部',
 'ビジネスソリューション本部 CRMソリューション部 インプリメンテーション2グループ': 'ビジネスソリューション本部 CRMソリューション部',
 'サービス＆サポート本部': 'サービス＆サポート本部',
 'サービス＆サポート本部 サービス＆サポート本部付': 'サービス＆サポート本部 サービス＆サポート本部付',
 'サービス＆サポート本部 サービスインテグレーション部': 'サービス＆サポート本部 サービスインテグレーション部',
 'サービス＆サポート本部 サービスインテグレーション部 プロダクトデベロップメントグループ': 'サービス＆サポート本部 サービスインテグレーション部',
 'サービス＆サポート本部 サービスインテグレーション部 サービスインテグレーショングループ': 'サービス＆サポート本部 サービスインテグレーション部',
 'サービス＆サポート本部 クラウドサービスセンター サービスオペレーション1グループ': 'サービス＆サポート本部 クラウドサービスセンター',
 'サービス＆サポート本部 クラウドサービスセンター サービスオペレーション2グループ': 'サービス＆サポート本部 クラウドサービスセンター',
 'サービス＆サポート本部 マネージドサービスセンター': 'サービス＆サポート本部 マネージドサービスセンター',
 'サービス＆サポート本部 マネージドサービスセンター Security Operation G': 'サービス＆サポート本部 マネージドサービスセンター',
 'サービス＆サポート本部 マネージドサービスセンター サービスオペレーショングループ': 'サービス＆サポート本部 マネージドサービスセンター',
 'サービス＆サポート本部 マネージドサービスセンター テクニカルアカウントマネジメントグループ': 'サービス＆サポート本部 マネージドサービスセンター',
 'サービス＆サポート本部 マネージドサービスセンター 更新サービスグループ': 'サービス＆サポート本部 マネージドサービスセンター',
 'サービス＆サポート本部 エンドポイントサービスセンター': 'サービス＆サポート本部 エンドポイントサービスセンター',
 'サービス＆サポート本部 エンドポイントサービスセンター エンドポイントサービス1グループ': 'サービス＆サポート本部 エンドポイントサービスセンター',
 'サービス＆サポート本部 エンドポイントサービスセンター エンドポイントサービス2グループ': 'サービス＆サポート本部 エンドポイントサービスセンター',
 'サービス＆サポート本部 ITサービスセンター': 'サービス＆サポート本部 ITサービスセンター',
 'サービス＆サポート本部 ITサービスセンター ITサービス1グループ': 'サービス＆サポート本部 ITサービスセンター',
 'サービス＆サポート本部 ITサービスセンター ITサービス2グループ': 'サービス＆サポート本部 ITサービスセンター',
 'サービス＆サポート本部 ITサービスセンター トレーニンググループ': 'サービス＆サポート本部 ITサービスセンター',
 'サービス＆サポート本部 沖縄事業所': 'サービス＆サポート本部 沖縄事業所',
 'サービス＆サポート本部 沖縄事業所 オペレーションセンター': 'サービス＆サポート本部 沖縄事業所',
 'サービス＆サポート本部 沖縄事業所 インサイドセールスセンター': 'サービス＆サポート本部 沖縄事業所',
 'サービス＆サポート本部 北海道事業所 北海道事業所付': 'サービス＆サポート本部 北海道事業所',
 'サービス＆サポート本部 北海道事業所 第1技術グループ': 'サービス＆サポート本部 北海道事業所',
 'サービス＆サポート本部 北海道事業所 第2技術グループ': 'サービス＆サポート本部 北海道事業所',
 'サービス＆サポート本部 九州事業所': 'サービス＆サポート本部 九州事業所',
 'サービス＆サポート本部 九州事業所 第1技術グループ': 'サービス＆サポート本部 九州事業所',
 'サービス＆サポート本部 九州事業所 第2技術グループ': 'サービス＆サポート本部 九州事業所',
 '技術管理室': '技術管理室',
 '技術管理室 業務管理': '技術管理室',
 # 以下海外
 'JBS USA': 'JBS USA',
 'JBS USA Sales': 'JBS USA Sales',
 'Administration Division': 'Administration Division',
 'Administration Division Administration': 'Administration Division',
 'Administration Division HR': 'Administration Division',
 'Administration Division Procurement': 'Administration Division',
 'Delivery Division': 'Delivery Division',
 'Delivery Division ソリューション部 ERP': 'Delivery Division',
 'Delivery Division Cloud and Network Solution_SI': 'Delivery Division',
 'Delivery Division Business Application and Development_SOL': 'Delivery Division',
 'Delivery Division NY': 'Delivery Division',
 'JBSAP': 'JBSAP',
 'JBS上海': 'JBS上海',
 '技術部 電装MS課': '技術部',
 '技術部 MSInfra開発課': '技術部',
 '技術部 BP課': '技術部',
 'サービス部 営業課': 'サービス部',
 'サービス部 総務人事課': 'サービス部',
 'JBSメキシコ': 'JBSメキシコ'
  }

# 2018/10からさかのぼりたい

In [45]:
 def change_dept(x):
    y = ' '.join(x)
    if y in DEPT_DICT:
        return DEPT_DICT[y]
    else:
        return None

In [46]:
tmp2 = itsDF['Organization'].str.split(' ', expand=False)

In [47]:
dept_s = tmp2.apply(change_dept)

In [48]:
itsDF['Dept'] = dept_s

In [49]:
itsDF.loc[itsDF.loc[:, 'CompanyName'] == 'JBSメキシコ','Div'] = itsDF.loc[itsDF.loc[:,'CompanyName'] == 'JBSメキシコ','Div'].map(lambda s: 'MEX-' + s)

In [50]:
itsDF.loc[itsDF['CompanyName'] == 'JBS USA', 'Dept'] = itsDF.loc[itsDF['CompanyName'] == 'JBS USA', 'Dept'].map(lambda s: 'USA-' + s)

In [51]:
itsDF.loc[itsDF['CompanyName'] == 'JBSシンガポール', 'Dept'] = itsDF.loc[itsDF['CompanyName'] == 'JBSシンガポール', 'Dept'].map(lambda s: 'SGP-' + s)

In [52]:
itsDF.loc[itsDF['CompanyName'] == 'JBS上海', 'Dept'] = itsDF.loc[itsDF['CompanyName'] == 'JBS上海', 'Dept'].map(lambda s: 'CHN-' + s)

In [53]:
itsDF.loc[itsDF['CompanyName'] == 'JBSメキシコ', 'Dept'] = itsDF.loc[itsDF['CompanyName'] == 'JBSメキシコ', 'Dept'].map(lambda s: 'MEX-' + s)

In [54]:
itsDF[itsDF['Dept'].isnull()]

,EmployeeNo,EmployeeName,AdAccount1,PersonId,EffectiveDate,ManagerId,LevelDesignation,Organization,CompanyName,HireDate,...,PreSales,Available,WorkingTime,OverTime,OnSite,hashedEmployeeNo,TimeZone,tmp_div,Div,Dept


In [55]:
itsDF['Dept'].unique()

array(['JBS', '内部監査室', '経営管理室', '財務経理部', '経営企画部', '品質管理室', 'リスク管理室',
       '人事部', '総務部', '業務部', '情報システム本部 インフラストラクチャサービス部',
       '情報システム本部 ビジネスサービス部', '社会システムソリューション室', '事業企画本部', '事業企画本部 事業企画本部付',
       '事業企画本部 総合企画部', '事業企画本部 海外事業推進室', '事業企画本部 人材開発部',
       '事業企画本部 MS統合サポートセンター', '事業企画本部 サービス企画部', '事業企画本部 サービス推進部',
       '事業企画本部 ビジネスインキュベーション部', '営業統括本部', '営業統括本部 営業統括本部付',
       '営業統括本部 第1営業本部', '営業統括本部 第1営業本部 営業1部', '営業統括本部 第1営業本部 営業2部',
       '営業統括本部 第2営業本部', '営業統括本部 第2営業本部 営業1部', '営業統括本部 第2営業本部 営業2部',
       '営業統括本部 第3営業本部', '営業統括本部 第3営業本部 第3営業本部付', '営業統括本部 第3営業本部 営業1部',
       '営業統括本部 第3営業本部 営業2部', '営業統括本部 第3営業本部 営業3部', '営業統括本部 パートナー営業本部',
       '営業統括本部 パートナー営業本部 営業部', '営業統括本部 中部・西日本営業本部',
       '営業統括本部 中部・西日本営業本部 中部営業部', '営業統括本部 中部・西日本営業本部 西日本営業部',
       '営業統括本部 中部・西日本営業本部 東京営業部', '営業統括本部 中部・西日本営業本部 北海道営業部',
       '営業統括本部 中部・西日本営業本部 九州営業部', '営業統括本部 ソリューション営業部', '営業統括本部 営業サポート室',
       'ITソリューション統括本部 ITソリューション統括本部付', 'ITソリューション統括本部 デジタルイノベーション推進室',
       'ITソリューション統括

### FunctionType

#### 先月とのDiv比較

In [56]:
itsDF['Div'].unique()

array(['JBS', '内部監査室', '経営管理室', '財務経理部', '経営企画部', '品質管理室', 'リスク管理室',
       '人事部', '総務部', '業務部', '情報システム本部', '社会システムソリューション室', '事業企画本部',
       '営業統括本部', '営業統括本部付', '第1営業本部', '第2営業本部', '第3営業本部', 'パートナー営業本部',
       '中部・西日本営業本部', 'ソリューション営業部', '営業サポート室', 'ITソリューション統括本部付',
       'デジタルイノベーション推進室', '金融システム本部', 'エンタープライズシステム本部', 'テクノロジーソリューション本部',
       'テクニカルパートナー本部', 'ビジネスソリューション本部', 'サービス＆サポート本部', '技術管理室', 'USA-JBS',
       'USA-Administration', 'USA-Delivery', 'SGP-JBSAP', 'CHN-JBS上海',
       'CHN-技術部', 'CHN-サービス部', 'MEX-MEX-JBSメキシコ'], dtype=object)

In [57]:
f_type_dic = {
'JBS': '経営',
'内部監査室': 'スタッフ',
'経営管理室': 'スタッフ',
'財務経理部': 'スタッフ',
'経営企画部': '企画',
'品質管理室': 'スタッフ',
'リスク管理室': 'スタッフ',
'人事部': 'スタッフ',
'総務部': 'スタッフ',
'業務部': 'スタッフ',
'情報システム本部': 'スタッフ',
'社会システムソリューション室': 'スタッフ',
'事業企画本部': '企画',
'営業統括本部': '営業',
'ITソリューション統括本部': '技術',
'ビジネスソリューション本部': '技術',
'サービス＆サポート本部': '技術',
'技術管理室': '技術',
'Administration': '海外',
'Delivery': '海外',
'JBSAP': '海外',
'JBS上海': '海外',
'技術部': '海外',
'サービス部': '海外',
'JBSメキシコ': '海外'
}

In [58]:
f_type_s = itsDF['tmp_div'].map(f_type_dic)

In [59]:
itsDF['FunctionType'] = f_type_s

In [60]:
itsDF[itsDF['FunctionType'].isnull()]

,EmployeeNo,EmployeeName,AdAccount1,PersonId,EffectiveDate,ManagerId,LevelDesignation,Organization,CompanyName,HireDate,...,Available,WorkingTime,OverTime,OnSite,hashedEmployeeNo,TimeZone,tmp_div,Div,Dept,FunctionType


# 旧JBSTの社員フラグ

In [61]:
itsDF['T'] = False

In [62]:
itsDF.loc[(itsDF['EmployeeNo'] >= 8000) & (itsDF['EmployeeNo'] < 40000), 'T'] = True

In [63]:
itsDF.loc[itsDF['EmployeeNo'] > 7000, ['EmployeeNo', 'T']]

,EmployeeNo,T
10,8001,True
11,7065,False
29,51392,False
33,50757,False
34,50758,False
35,51380,False
39,9546,True
45,8559,True
47,9636,True
51,8010,True


### カラム名変更

In [64]:
itsDF.columns

Index(['EmployeeNo', 'EmployeeName', 'AdAccount1', 'PersonId', 'EffectiveDate',
       'ManagerId', 'LevelDesignation', 'Organization', 'CompanyName',
       'HireDate', 'EntryType', 'EmploymentType', 'CustomerCharge', 'PreSales',
       'Available', 'WorkingTime', 'OverTime', 'OnSite', 'hashedEmployeeNo',
       'TimeZone', 'tmp_div', 'Div', 'Dept', 'FunctionType', 'T'],
      dtype='object')

#### 情シス提供のデータのカラム名とWpAで定義したカラム名の相違の吸収

In [65]:
itsDF.rename(columns={'EmploymentType': 'Employmenttype',
                            'OverTime': 'Overtime',
                           'EmployeeNo': 'No'}, inplace=True)

In [66]:
itsDF.head(3)

,No,EmployeeName,AdAccount1,PersonId,EffectiveDate,ManagerId,LevelDesignation,Organization,CompanyName,HireDate,...,WorkingTime,Overtime,OnSite,hashedEmployeeNo,TimeZone,tmp_div,Div,Dept,FunctionType,T
0,1,牧田 幸弘,ymakita,makita@jbs.com,07/01/2019,NaN,代表取締役社長,JBS,JBS,01/01/1991,...,0.0,0.0,0,856f4e0216a474d7d13f5b680e326e5f46bc181dfb68e2...,Tokyo Standard Time,JBS,JBS,JBS,経営,False
1,7,牧田 和也,kmakita,kazuya.makita@jbs.com,07/01/2019,makita@jbs.com,取締役副社長,JBS,JBS,10/11/1993,...,0.0,0.0,0,b1743d531fa84a32613ea5f71342e26011130b0ab5ccd1...,Tokyo Standard Time,JBS,JBS,JBS,経営,False
2,6,和田 行弘,ywada,yukihiro.wada@jbs.com,07/01/2019,makita@jbs.com,取締役常務執行役員,JBS,JBS,05/11/1992,...,0.0,0.0,0,de726a90da3a4d7f9cf4b671fae4c9cff2478ebec17861...,Tokyo Standard Time,JBS,JBS,JBS,経営,False


### Evaluationカラムの追加

In [67]:
itsDF['Evaluation'] = np.nan

### 必要なカラムのみ選択

In [68]:
resultDF = itsDF[['PersonId', 
                   'EffectiveDate', 
                   'ManagerId', 
                   'Organization',
                   'FunctionType',
                   'TimeZone', 
                   'HireDate',
                   'Div',
                   'Dept',
                   'EntryType',
                   'Employmenttype',
                   'CompanyName',
                   'CustomerCharge',
                   'PreSales',
                   'Available',
                   'Overtime',
                   'OnSite', 
                   'LevelDesignation',
                   'No',
                   'Evaluation',
                   'T'
                 ]]

In [69]:
resultDF.head(3)

,PersonId,EffectiveDate,ManagerId,Organization,FunctionType,TimeZone,HireDate,Div,Dept,EntryType,...,CompanyName,CustomerCharge,PreSales,Available,Overtime,OnSite,LevelDesignation,No,Evaluation,T
0,makita@jbs.com,07/01/2019,NaN,JBS,経営,Tokyo Standard Time,01/01/1991,JBS,JBS,中途,...,JBS,0.0,0.0,0.0,0.0,0,代表取締役社長,1,NaN,False
1,kazuya.makita@jbs.com,07/01/2019,makita@jbs.com,JBS,経営,Tokyo Standard Time,10/11/1993,JBS,JBS,中途,...,JBS,0.0,0.0,0.0,0.0,0,取締役副社長,7,NaN,False
2,yukihiro.wada@jbs.com,07/01/2019,makita@jbs.com,JBS,経営,Tokyo Standard Time,05/11/1992,JBS,JBS,中途,...,JBS,0.0,0.0,0.0,0.0,0,取締役常務執行役員,6,NaN,False


In [70]:
resultDF.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8')

In [71]:
resultDF.count()

PersonId            2985
EffectiveDate       2985
ManagerId           2971
Organization        2985
FunctionType        2985
TimeZone            2985
HireDate            2985
Div                 2985
Dept                2985
EntryType           2985
Employmenttype      2985
CompanyName         2985
CustomerCharge      2985
PreSales            2985
Available           2985
Overtime            2985
OnSite              2985
LevelDesignation    2985
No                  2985
Evaluation             0
T                   2985
dtype: int64